In [13]:
def analyse_my_stock_portfolio2(tickers, timeframe, start_date, end_date, weights, simulate, inv_amt, num_sim, num_trade_days):
   
    # Import packages required
    import os
    import pandas as pd
    import numpy as np
    import datetime as dt
    import matplotlib.pyplot as plt 
    from dotenv import load_dotenv
    import alpaca_trade_api as tradeapi
    import panel as pn
    import numpy as np
    import cufflinks as cf
    from IPython.display import display
    pn.extension('plotly')
    import plotly.express as px
    import hvplot.pandas
    from dotenv import load_dotenv
    from MCForecastTools import MCSimulation
    import holoviews as hv
    #List of Cufflinks Themes :  ['ggplot', 'pearl', 'solar', 'space', 'white', 'polar', 'henanigans']
    cf.set_config_file(theme='solar',sharing='public',offline=True)
    load_dotenv()     
    %matplotlib inline        
    
                                 ####################################################################### Data Setup #######################################################################
    
    #Define variables
    tickers = tickers
    tf = timeframe
    start_date = pd.Timestamp(start_date, tz='America/New_York').isoformat()
    end_date = pd.Timestamp(end_date, tz='America/New_York').isoformat()
    num_sim = num_sim
    wt=weights
    num_trade_days=num_trade_days   
    simulate = simulate
    
    #Read in alpaca keys
    alpaca_key = os.getenv('ALPACA_API_KEY')
    alpaca_secret_key = os.getenv('ALPACA_API_SECRET_KEY')
    
    print(f"Your alpaca key is {type(alpaca_key)}.")
    print(f"Your alpaca secret key is {type(alpaca_secret_key)}.")
    print()
    
    #Set up Alpaca trade api
    api = tradeapi.REST(alpaca_key, alpaca_secret_key, api_version='V2')
    
    #Define parameters to get stock data from Alpaca api
    tickers_df = api.get_barset(tickers, timeframe=tf, start=start_date, end=end_date, limit=1000).df
    
    #Calc daily returns
    close_prices = tickers_df.loc(axis=1)[:,'close']
         
        
                                 ####################################################################### ANALYSES #######################################################################
            

    close_prices = tickers_df.loc(axis=1)[:,'close']

    daily_returns = close_prices.pct_change()
    
    daily_returns2 = close_prices.pct_change()
    daily_returns2.columns = daily_returns2.columns.droplevel(1)
    daily_returns2.dropna(inplace=True)
    
    correlation = daily_returns.corr()
    correlation.columns = correlation.columns.droplevel(1)
    correlation.index = correlation.index.droplevel(1)    
    
    portfolio_cum_return = (1+daily_returns).cumprod()  
    
    portfolio_cum_return2 = (1+daily_returns).cumprod()
    portfolio_cum_return2.columns = portfolio_cum_return2.columns.droplevel(1)
    portfolio_cum_return2.dropna(inplace=True)

    roll_std = daily_returns.rolling(window=5).std()
    
    roll_std2 = daily_returns.rolling(window=5).std()
    roll_std2.columns = roll_std2.columns.droplevel(1)
    roll_std2.dropna(inplace=True)
    
    sharpe_ratio = (daily_returns.mean()*252) / (daily_returns.std()*np.sqrt(252))
    
    new = []

    for i in range(len(tickers)):
        ticker_df = pd.concat([daily_returns.iloc[:, i], portfolio_cum_return.iloc[:, i], roll_std.iloc[:, i]], axis=1, join='inner')
        ticker_df["Ticker"] = (f"{tickers[i]}")
        ticker_df = ticker_df.droplevel(1, axis=1)
        ticker_df.columns = ["daily_returns", "cum_ret", "roll_std", "ticker"]
        new.append(ticker_df)

    final_df = pd.concat(new, axis=0).dropna()
    display(final_df.head(10))
    
    MC = MCSimulation(portfolio_data=tickers_df, weights=wt, num_simulation=num_sim, num_trading_days=num_trade_days)
    print()
    display(MC.portfolio_data.head())
    print()     
    
    
    if simulate == 'yes':
        #Run Monte-Carlo simulation to determine cumulative returns
        print("Running simulation..........................")
        print()
        print()
        print()
        
        #Summarise cumulative returns
        MC_summary = MC.summarize_cumulative_return()
        print()
        print(f"Here is the summary of the Monte-Carlo simulation {MC_summary}")
        
        #Extract confidence interval to determine range for future cumulative returns
        ci_lower = round(MC_summary[8] * inv_amt, 2)
        ci_upper = round(MC_summary[9] * inv_amt, 2)
    
        print()
        print(f"If you were to invest ${inv_amt} in {tickers[0:]} with a weighting of {wt}, then your invested amount could range from ${ci_lower} to ${ci_upper} after {num_trade_days} trading days or {num_trade_days/252} years.")
        print()
        print()
        print()

    elif simulate == 'no':
            pass
    else: 
            pass
        
        
                                         ####################################################################### Plots #######################################################################
            
   
    def daily_returns():
        #hv.renderer('bokeh').theme = 'dark_minimal'
        plot1 = daily_returns2.iplot(title="Daily Returns")
                                                                           
                
        return plot1



    def cum_ret():
        plot2 = portfolio_cum_return2.iplot(title="Cumulative Returns")
                                
        return plot2



    def roll_std():
        plot3 = roll_std2.iplot(title="5-Day rolling std dev")
        
                                
        return plot3
    
    
    def correlation_plot():
        plot4 = correlation.iplot(kind='heatmap')
        
        return plot4
    
    
    
                                             ####################################################################### Dashboard #######################################################################

    
    set_panels1 = pn.Column(daily_returns(),
                            cum_ret())
    
    set_panels2 = pn.Column(roll_std(),
                            correlation_plot())
    
    title = pn.pane.Markdown("### **Here is a breakdown of your stock portfolio analysis**")
    
    portoflio_analysis_dashboard_tabs = pn.Tabs(('Stock portfolio analysis', pn.Row(set_panels1, set_panels2)))
    
    portoflio_analysis_dashboard = pn.Column(pn.Row(title), portoflio_analysis_dashboard_tabs)
    
    portoflio_analysis_dashboard.servable()
    
    return portoflio_analysis_dashboard

In [14]:
tickers = ['FB', 'GOOG', 'AAPL', 'AMZN', 'AMD']

timeframe = '1D'

start_date = "2017-1-1"

end_date = '2021-1-1'

weights = [0.20, 0.20, 0.20, 0.20, 0.20] 

simulate = 'no'

inv_amt = 20000

num_sim = 50

num_trade_days = 50

analyse_my_stock_portfolio2(tickers, timeframe, start_date, end_date, weights, simulate, inv_amt, num_sim, num_trade_days)

#panel serve --show .\FinancialPlan_Viz.ipynb
#bokeh serve --show --port 5001/2 .\.ipynb file

Your alpaca key is <class 'str'>.
Your alpaca secret key is <class 'str'>.



,daily_returns,cum_ret,roll_std,ticker
time,,,,
2017-01-20 00:00:00-05:00,0.001837,1.006205,0.004089,FB
2017-01-23 00:00:00-05:00,0.000833,1.007044,0.003744,FB
2017-01-24 00:00:00-05:00,-0.000999,1.006038,0.001403,FB
2017-01-25 00:00:00-05:00,0.016087,1.022222,0.007329,FB
2017-01-26 00:00:00-05:00,0.000328,1.022558,0.007045,FB
2017-01-27 00:00:00-05:00,-0.001804,1.020713,0.007452,FB
2017-01-30 00:00:00-05:00,-0.001314,1.019371,0.007659,FB
2017-01-31 00:00:00-05:00,-0.002139,1.017191,0.007803,FB
2017-02-01 00:00:00-05:00,0.061418,1.079665,0.028034,FB


AAPL                                    \
                              open    high     low   close    volume   
time                                                                   
2017-01-12 00:00:00-05:00  118.895  119.30  118.21  119.25  22972978   
2017-01-13 00:00:00-05:00  119.110  119.62  118.81  119.03  20561275   
2017-01-17 00:00:00-05:00  118.340  120.24  118.22  119.99  28121739   
2017-01-18 00:00:00-05:00  120.000  120.50  119.71  119.97  18562058   
2017-01-19 00:00:00-05:00  119.400  120.09  119.37  119.77  18277861   

                                          AMD                          ...  \
                          daily_return   open     high    low   close  ...   
time                                                                   ...   
2017-01-12 00:00:00-05:00          NaN  10.98  11.0376  10.33  10.760  ...   
2017-01-13 00:00:00-05:00    -0.001845  10.79  10.8700  10.56  10.580  ...   
2017-01-17 00:00:00-05:00     0.008065  10.17  10.2300   9.78   9.815  ...   
2017-01-18 00:00:00-05:00    -0.000167   9.50  10.1000   9.42   9.880  ...   
2017-01-19 00:00:00-05:00    -0.001667   9.92  10.2500   9.75   9.780  ...   

                               FB                                   GOOG  \
                              low   close    volume daily_return    open   
time                                                                       
2017-01-12 00:00:00-05:00  124.80  126.64  16127037          NaN  807.14   
2017-01-13 00:00:00-05:00  127.37  128.33  21777028     0.013345  807.48   
2017-01-17 00:00:00-05:00  127.40  127.85  11700685    -0.003740  807.08   
2017-01-18 00:00:00-05:00  126.84  127.92  10714808     0.000548  805.81   
2017-01-19 00:00:00-05:00  127.45  127.55   9768484    -0.002892  805.12   

                                                                          
                               high     low   close  volume daily_return  
time                                                                      
2017-01-12 00:00:00-05:00  807.3900  799.17  806.34  875205          NaN  
2017-01-13 00:00:00-05:00  811.2244  806.69  807.80  695463     0.001811  
2017-01-17 00:00:00-05:00  807.1400  800.37  804.61  647136    -0.003949  
2017-01-18 00:00:00-05:00  806.2050  800.99  806.06  811921     0.001802  
2017-01-19 00:00:00-05:00  809.4800  801.80  802.12  537971    -0.004888  

[5 rows x 30 columns]

Column
    [0] Row
        [0] Markdown(str)
    [1] Tabs
        [0] Row
            [0] Column
                [0] Str(None)
                [1] Str(None)
            [1] Column
                [0] Str(None)
                [1] Str(None)